<a href="https://colab.research.google.com/github/apuas/nn-python/blob/main/ForVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!ls "/content/drive/My Drive/"

In [ ]:
import matplotlib
matplotlib.use("Agg")
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2
import os

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint



In [ ]:
ImagePaths = list(paths.list_images("/content/drive/My Drive/GoogleColab/dogs/"))
random.seed(42)
ImagePaths = ImagePaths[:1800] + list(paths.list_images("/content/drive/My Drive/GoogleColab/cats/"))
print(len(ImagePaths))
random.shuffle(ImagePaths)

data = []
labels = []

for imagepath in ImagePaths:
  image = cv2.imread(imagepath)
  image = cv2.resize(image, (32, 32)).flatten()
  data.append(image)
  label = imagepath.split(os.path.sep)[-2]
  if label == "cats":
    label = [1,0]
  else:
    label = [0,1]
  labels.append(label)

print(labels)  

data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

with open("/content/drive/My Drive/GoogleColab/Lesson_3/data.pickle", 'wb') as f:
  pickle.dump(data, f)
print("Data seved")

with open("/content/drive/My Drive/GoogleColab/Lesson_3/labels.pickle", 'wb') as f:
  pickle.dump(labels, f)
print("Labels seved")

In [ ]:
with open("/content/drive/My Drive/GoogleColab/Lesson_3/data.pickle", 'rb') as f:
  data = pickle.load(f)
print("Data loaded")

with open("/content/drive/My Drive/GoogleColab/Lesson_3/labels.pickle", 'rb') as f:
  labels = pickle.load(f)
print("Labels loaded")

(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.15,
                                                  random_state=42)

print("Dataset prepared")

Data loaded
Labels loaded
Dataset prepared


In [ ]:
model = Sequential() 
model.add(Dense(1024,input_shape=(3072,), activation='sigmoid'))
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))

INIT_LR = 0.01
opt = SGD(lr=INIT_LR)

model.compile(loss="binary_crossentropy",optimizer=opt,
              metrics=["accuracy"])  #categorial_crosentropy

print ("Model compiled")

Model compiled


In [ ]:
EPOCHS = 70

# checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/GoogleColab/Lesson_3/EasyNet_{epoch:2d}_{val_accuracy:.3f}.hdf5', verbose=1, period=10)
 checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/GoogleColab/Lesson_3/point.h5', verbose=1, save_best_only=True)
# checkpointer = ModelCheckpoint(filepath='/content/drive/My Drive/GoogleColab/Lesson_3/point.h5', verbose=1)


# model.fit(x_train, y_train, batch_size=128, epochs=20, verbose=0, validation_data=(X_test, Y_test), callbacks=[checkpointer])


H = model.fit(trainX, trainY, validation_data=(testX,testY),
              epochs=EPOCHS, batch_size=128,
              callbacks=[checkpointer])

print("Model trained")

In [ ]:
predictions = model.predict (testX, batch_size = 32)
print(predictions)

print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=("cats","dogs")))

N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="vall_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Results")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy/")
plt.legend()
plt.savefig("/content/drive/My Drive/GoogleColab/Lesson_3/Loss.png")
model.save("/content/drive/My Drive/GoogleColab/Lesson_3/EasyNet.model")

print("End")



[[0.58963454 0.41036543]
 [0.4803841  0.5196159 ]
 [0.59001416 0.4099858 ]
 ...
 [0.52583    0.47416997]
 [0.4504728  0.5495272 ]
 [0.46542823 0.53457177]]
              precision    recall  f1-score   support

        cats       0.54      0.52      0.53       263
        dogs       0.55      0.56      0.56       271

    accuracy                           0.54       534
   macro avg       0.54      0.54      0.54       534
weighted avg       0.54      0.54      0.54       534

End
